In [64]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB10 --baud=115200 
Ready.


In [61]:
%sendtofile config.txt

wifiname     DoESLiverpool
wifipassword decafbad00
mqttbroker   10.0.100.1
pinled       -2
wifialt      BV6000,bubblino,192.168.43.1

Sent 5 lines (132 bytes) to config.txt.


In [65]:
%sendtofile --source stdmqttas.py
%sendtofile --quiet --source utils.py

Sent 61 lines (2254 bytes) to stdmqttas.py.
Sent 32 lines (969 bytes) to utils.py.


In [67]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB8 --baud=115200 
Ready.


In [68]:
%sendtofile main.py

import time, sys, machine, itertools
import uasyncio as asyncio
from stdmqttas import fconfig, config, mqttconnecttask, callbackcmdtask, flashledconnectedtask
from mqtt_as import MQTTClient

from stdmqttas import pinled, flashpinled, shortmac

topicstem = "%s/%s" % (sys.platform, shortmac)
print("topicstem", topicstem)
import network
network.WLAN().active(0)  # disable the connection at startup

i2c = machine.I2C(scl=machine.Pin(5), sda=machine.Pin(4), freq=60000)  # D1 D2
flashpinled(5, 300, 300)

topicMLX6015 = topicstem+"/MLX6015"
topicstatus = topicstem+"/status"
topiccmd = topicstem+"/cmd"
topicreply = topicstem+"/reply"

def crc8(data):
    c = 0
    for d in data:
        c = c ^ d
        for i in range(8):
            if c & 0x80:
                c = (c << 1) ^ 7
            else:
                c = c << 1
        c = c & 0xFF
    return c
        
def readtemp(reg):
    try:
        k = i2c.readfrom_mem(0x5B, reg, 3)
    except OSError:
        return -999
    kcrc = crc8([0x5B<<1, reg, (0x5B<<1)+1, k[0], k[1]])
    return (k[0]+k[1]*256)*0.02 - 273.15  if k[2] == kcrc  else -999

async def topicMLX6015task(client):
    print("publishing on", topicMLX6015)
    for i in itertools.count():
        tempAmbient, tempIR = readtemp(0x26), readtemp(0x27)
        print(tempAmbient, tempIR)
        if client.isconnected():
            await client.publish(topicMLX6015, "%f %f"%(tempAmbient, tempIR))
        await asyncio.sleep_ms(500)
        
def callbackcmd(topic, msg, retained):
    aloop.create_task(callbackcmdtask(client, topicreply, msg))

async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    await client.publish(topicstatus, ipnumber, retain=True)
    await client.subscribe(topiccmd, 1)
    
config['subs_cb'] = callbackcmd
config['connect_coro'] = onconnecttask
config['will'] = (topicstatus, "offline", True)
client = MQTTClient(config)
client.DEBUG = True
  
aloop = asyncio.get_event_loop()
aloop.create_task(topicMLX6015task(client))
aloop.create_task(mqttconnecttask(client))
aloop.create_task(flashledconnectedtask(client))
aloop.run_forever()


Sent 68 lines (2116 bytes) to main.py.
